# 7章 プロのようにデータを操る
---
この章では、データを自在に操るためのさまざまなテクニックを学ぶ。その大半は次の組み込みデータ型に関係するものだ。

- 文字列
  - Unicode文字のシーケンス。テキストデータで使われる。
- バイトとバイト列
  - 8ビット整数のシーケンス。バイナリデータで使われる。

## 7.1 文字列
---
テキスト（文字列）はほとんどの読者にとってもっとも馴染み深いデータだろう。そこで、Python文字列の強力な機能から説明を始めることにしよう。

### 7.1.1 Unicode
---
本書の今までのテキスト関連のサンプルでは、すべてASCIIを使ってきた。ASCIIは、コンピュータが冷蔵庫ほどの大きさで、計算が少し得意なだけだった1960年代に定義されたものだ。コンピュータの記憶の基本単位は**バイト**で、8個の**ビット**を使って256種類の一意な値を表現出来る。さまざまな理由から、ASCIIは7ビット（128種類の一意な値）しか使っていない。26種類の大文字、26種類の小文字、10種類の数字、いくつかの記号、いくつかの空白文字、そしていくつかの表示されない制御コードが含まれている。

残念ながら、世界にはASCIIで表現できる以上の文字がある。夕食にホットドッグは食べられても、カフェ（café）でGewürztraminerを飲むことはできなくなってしまう。文字と記号を増やすために、さまざまな試みがなされてきた。ときどき、それを見かけることもあるだろう。そのような試みからふたつだけを紹介しておこう。

- Latin-1、あるいはISO 8859-1
- Windowsコードページ1252

これらは8ビットをフルに使っているが、それでも十分ではないことがある。特に、ヨーロッパ以外の言語が必要になったときにはそうだ。**Unicode**は、世界の言語のすべての文字と数学、その他の分野の記号を定義しようという発展途上の国際基準である。

> Unicodeは、プラットフォーム、プログラム、言語にかかわらず、すべての文字に一意な番号を与える。
> The Unicode Consortium

[Unicodeのコード一覧ページ](http://unicode.org/charts/)には、現在定義されているすべてのキャラクターセットとそのイメージへのリンクが含まれている。最新バージョン（10.0）は、12万字を超える文字、記号を定義しており、それぞれに一意な名前とID番号を与えている。文字は、**面**と呼ばれる8ビットセットに分割される。最初の256面は、基本多言語面である。詳しくは、[WikipediaのUnicode面についてのページ（日本語）](http://bit.ly/2EIoHKv)、[WikipediaのUnicode面についてページ（英語）](http://bit.ly/2Flht0e)を参照していただきたい。

#### 7.1.1.1
---
Python3の文字列はUnicode文字列で、バイト列ではない。Python2から3への移行で、これがもっとも大きな変更だ。Python2は、1バイト文字列とUnicode文字列を区別していた。

文字のUnicode IDまたは名前を知っている場合、Python文字列でそれを使うことができる。例をいくつか示そう。

- \uの後ろに4個の16進数字を続けたものは、256の基本多言語面のどれかに含まれる文字に対応する。最初の2桁は面番号（00からFFまで）、後半の2桁はその面のなかでの文字のインデックスを表す。面00はASCIIであり、面内での文字の位置もASCIIと同じになっている。
- 上位面の文字は、基本多言語面の文字よりも多くのビットを必要とする。Pythonのエスケープシーケンスは、\Uの後ろに8個の16進文字を続けたものである。左端の数字は0でなければならない。
- すべての文字について、\N{name}を使えば、標準の**名前**を通じてひとつの文字を指定できる。名前のリストは、[Unicode文字名索引](http://unicode.org/charts/charindex.html)にまとめられている。

Pythonのunicodedataモジュールには、双方向の変換関数が含まれている。

- lookup()
  - 名前（大文字と小文字を区別しない）を与えると、Unicode文字が返される。
- name()
  - Unicode文字を与えると、大文字の名前が返される。
  
次のサンプルでは、Unicode文字を引数とするテスト関数を書いている。文字から名前を引き出し、名前から文字を引き出している（最初の文字と一致するはずだ）。

In [7]:
def unicode_test(value):
    import unicodedata
    name = unicodedata.name(value)
    value2 = unicodedata.lookup(name)
    print('value="%s", name="%s", value2="%s"' % (value, name, value2))

いくつかの文字を試してみよう。最初はごく平凡なASCII文字だ。

In [8]:
unicode_test('A')

value="A", name="LATIN CAPITAL LETTER A", value2="A"


次はASCIIの記号である。

In [9]:
unicode_test('$')

value="$", name="DOLLAR SIGN", value2="$"


Unicodeの通貨記号を見てみよう。

In [10]:
unicode_test('\u00a2')

value="¢", name="CENT SIGN", value2="¢"


Unicodeの別の通貨記号だ。

In [12]:
unicode_test('\u20ac')

value="€", name="EURO SIGN", value2="€"


このテストをしていて問題になりそうなことは、テキストの表示に使っているフォントだけだ。すべてのフォントがすべてのUnicode文字のイメージを持っているわけではなく、イメージのない文字に対してはそのことを表す代替記号を表示する。たとえば、次に示すのは、Unicodeの「雪だるま」記号である。dinggatフォントに含まれている記号と同じようなものだ。

In [13]:
unicode_test('\u2603')

value="☃", name="SNOWMAN", value2="☃"


Pythonの文字列にCaféという単語を保存したいときにはどうすればよいだろうか。ファイルやウェブサイトからコピーアンドペーストしてうまくいくように祈るのもひとつの方法だ。

In [15]:
place = 'café'
place

'café'

これでうまくいったのは、テキストのためにUTF-8エンコーディング（すぐあとで説明する）を使っているソースからコピーアンドペーストしたからである。

最後のéの文字はどのようにして指定すればよいのだろうか。[Eの文字列検索](http://www.unicode.org/charts/charindex.html#E)を見ると、E WITH ACUTE, LATIN SMALL LETTERという名前は00E9という値を持っていることがわかる。今いじってみたname()、lookup()関数でチェックしてみよう。まず、コードから名前を調べてみる。

In [18]:
import unicodedata
unicodedata.name('\u00E9')

'LATIN SMALL LETTER E WITH ACUTE'

次に、名前からコードを調べてみる。

In [23]:
unicodedata.lookup('E WITH ACUTE, LATIN SMALL LETTER')

KeyError: "undefined character name 'E WITH ACUTE, LATIN SMALL LETTER'"

> Unicode文字列検索ページは、ソート、表示に適したように整形されている。実際のUnicode名（Pythonが使っているもの）に変換するには、カンマを削除し、カンマの後ろの部分を前に移動する。そこで、E WITH ACUTE, LATIN SMALL LETTERは、LATIN SMALL LETTER E WITH ACUTEになる。

In [22]:
unicodedata.lookup('LATIN SMALL LETTER E WITH ACUTE')

'é'

これで、コードから名前によってcaféという文字列を指定できるようになった。

In [25]:
place = 'caf\u00e9'
place

'café'

In [26]:
place = 'caf\N{LATIN SMALL LETTER E WITH ACUTE}'
place

'café'

上のコードでは、文字列にéを直接挿入していたが、文字を追加して文字列を組み立てていくこともできる。

In [27]:
u_umlaut = '\N{LATIN SMALL LETTER U WITH DIAERESIS}'
u_umlaut

'ü'

In [28]:
drink = 'Gew' + u_umlaut + 'rstraminer'
print('Now I can finally have my', drink, 'in a', place)

Now I can finally have my Gewürstraminer in a café


文字列のlen関数は、バイト数ではなく、Unicodeの**文字数**を数える。

In [29]:
len('$')

1

In [30]:
len('\U0001f47b')

1

#### 7.1.1.2 UTF-8によるエンコード、デコード
---
通常の文字列処理をしているときには、Pythonが個々のUnicode文字をどのように格納しているかについて心配する必要はない。

しかし、外部の世界との間でデータをやり取りするときには、次のふたつのことが必要になる。

- 文字列をバイト列に**エンコード**する手段
- バイト列を文字列に**デコード**する手段

Unicodeが64,000文字よりも少なければ、個々のUnicode文字IDを2バイトに格納できたところだが、実際にはもっと多い。3，4バイトあればすべてのIDをエンコードできるが、それではごく普通の文字列のためにメモリやディスクで必要な容量が3、4倍に増えてしまう。

Ken ThompsonとRob Pikeと言えば、Unixデベロッパーにはおなじみの名前だろう。動的エンコード方式のUTF-8は、ある晩彼らがニュージャージーの食堂の一角で設計したものである。UTF-8は、個々のUnicode文字のために1バイトから4バイトを使っている。

- ASCIIは1バイト
- ほとんどのローマ字系言語には2バイト（ただしキリル文字は除く）
- 基本多言語面のその他については3バイト
- 一部のアジアの言語、記号を含むその他については4バイト

UtF-8はPython、Linux、HTMLでは標準的なテキストエンコーディングであり、非常によく機能している。コード全体を通じてUTF-8エンコーディングを使うなら、さまざまなエンコーディングの間を行き来するよりもはるかに仕事が楽になる。

> ウェブページなどの他のソースからコピーアンドペーストでPython文字列を作るときには、ソースがUTF-8形式でエンコードされていることを確かめなければならない。Latin-1やWindows 1252でエンコードされたテキストをPython文字列にコピーしている例は**非常に**よく見かける。すると、あとで無効なバイトシーケンスだとして例外を引き起こすことになる。

#### 7.1.1.3 エンコーディング
---
**文字列**をエンコードして**バイト**にする。文字列のencode()関数の第1引数は、エンコーディング名だ。表7−1にまとめられているものから選べる

表7−1 エンコーディング

| エンコーディング名 | 意味 |
|:----------------|:----|
| 'ascii'         | 古き良き7ビットASCII |
| 'utf-8'         | 8ビット可変長エンコーディング。ほとんどかならずこれを使うことになる |
| 'latin-1'       | ISO 8859-1とも呼ばれているもの |
| 'cp-1259'       | 一般的なWindowsエンコーディング |
| 'unicode-escape' | Python Unicodeリテラル方式。 \uxxxxまたは\Uxxxxxxxx |

どんなものでもUTF-8としてエンコードすることができる。snowmanという変数にUnicode文字列'\u2603'を代入してみよう。

In [31]:
snowman = '\u2603'

Python内部に格納するために何バイトかかったかにかかわらず、snowmanは、1文字のPython Unicode文字列だ。

In [32]:
len(snowman)

1

次に、このUnicode文字をバイトシーケンスにエンコードしてみよう。

In [33]:
ds = snowman.encode('utf-8')

先ほども触れたように、UTF-8は可変長エンコーディングだ。この場合、Unicodeのたったひとつの文字をエンコードするために3バイトを使っている。

In [34]:
len(ds)

3

In [35]:
ds

b'\xe2\x98\x83'

今度は、len()がバイト数(3)を返しているが、それはdsがbytes変数（「7.2.1 バイトとバイト列」参照）だからだ。

UTF-8以外のエンコーディングも使えるが、そのエンコーディングでUnicode文字列が処理出来ない場合はエラーが返される。たとえば、asciiエンコーディングを使うと、Unicode文字が有効なASCII文字になっている場合でなければエラーになる。

In [36]:
ds = snowman.encode('ascii')

UnicodeEncodeError: 'ascii' codec can't encode character '\u2603' in position 0: ordinal not in range(128)

encode関数は、エンコード例外を起こしにくくするための第2引数を持っている。デフォルト値は、今までの例のような動作をする。'strict'で、ASCII以外の文字が使われているとUnicodeEncodeErrorを起こす。しかし、ほかの値も指定できる。'ignore'を使えば、エンコードできないものを捨ててしまう。

In [37]:
snowman.encode('ascii', 'ignore')

b''

'replace'を使えば、エンコードできない文字を?に置き換える。

In [38]:
snowman.encode('ascii', 'replace')

b'?'

'backslashreplace'を使えば、unicode-escape形式のPython Unicode文字列を生成する。

In [39]:
snowman.encode('ascii', 'backslashreplace')

b'\\u2603'

Unicodeエスケープシーケンスの表示可能バージョンが必要なら、これを使うことになるだろう。

次のコードは、ウェブページで使えるエンティティの文字列を生成する。

In [40]:
snowman.encode('ascii', 'xmlcharrefreplace')

b'&#9731;'

#### 7.1.1.4 デコーディング
---
バイト列を**デコード**してUnicode文字列にする。なんらかの外部ソース（ファイル、データベース、ウェブサイト、ネットワークAPIなど）からテキストを取り出したとき、そのテキストはバイト列としてエンコードされている。難しいのは、実際にどのエンコーディングが使われているのかを知ることだ。それがわからなければ、エンコードの「逆」をやってUnicode文字列を得ることはできない。

しかし、バイト列自体のなかには、どのエンコーディングが使われているかを教えてくれるものはない。ウェブサイトからのコピーアンドペーストの危険性については先ほど触れた。古き良きASCII文字が表示されるべきところに奇妙な文字が表示されているウェブサイトを見かけたことはあるだろう。

では、placeという変数で、値が'café'のUnicode文字列を作ってみよう。


In [41]:
place = 'caf\u00e9'
place

'café'

In [42]:
type(place)

str

これをUTF-8形式でエンコードしてplace_bytesというbytes変数に格納しよう。

In [43]:
place_bytes = place.encode('utf-8')
place_bytes

b'caf\xc3\xa9'

In [44]:
type(place_bytes)

bytes

place_bytesが5バイトだということに注意しよう。最初の3バイトはASCIIと同じ（UTF-8の長所）で、最後の2バイトが'é'をエンコードしている。では、バイト列をUnicode文字列にデコードしてみよう。

In [47]:
place2 = place_bytes.decode('utf-8')
place2

'café'

これがうまく動作したのは、UTF-8にエンコードし、UTF-8からデコードしたからだ。ほかのエンコーディングからのデコードを指示したらどうなるだろうか。

In [48]:
place3 = place_bytes.decode('ascii')

UnicodeDecodeError: 'ascii' codec can't decode byte 0xc3 in position 3: ordinal not in range(128)

ASCIIデコーダは、oxc3というバイト値がASCIIでは無効なので例外を投げている。8ビットキャラクタセットのエンコーディングでは、
128（16進80）から255（16進FF）までの間に有効な値を持つものがあるが、それはUTF-8とは異なる値だ。

In [49]:
place4 = place_bytes.decode('latin-1')
place4

'cafÃ©'

In [50]:
place5 = place_bytes.decode('windows-1252')
place5

'cafÃ©'

これではダメである。

ここから得られる教訓は、可能な限りUTF-8エンコーディングを使えということだ。UTF-8なら正しく動作し、どこでもサポートされているし、すべてのUnicode文字を表現でき、すばやくデコード、エンコードできる。

#### 7.1.1.5 詳しく学びたい人のために
---
エンコード、デコードについてもっと学びたい場合は、次のリンクが役に立つだろう。

- [Unicode HOWTO](https://docs.python.org/ja/3/howto/unicode.html)
- [Pragmatic Unicode](https://nedbatchelder.com/text/unipain.html)
- [The Absolute Minimum Every Software Developer Absolutely, Positively Must Know About Unicode and Charactor Sets \(No Excuses!\)](https://www.joelonsoftware.com/2003/10/08/the-absolute-minimum-every-software-developer-absolutely-positively-must-know-about-unicode-and-character-sets-no-excuses/)